In [1]:
from numpy.random.bit_generator import SeedlessSeedSequence
import tensorflow as tf
import os
import numpy as np
import random
from sklearn.utils import shuffle
#from tqdm import tqdm #Provides a progress bar while the images are being resized
from skimage.io import imshow, imread
from skimage.transform import resize
from matplotlib import pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import tifffile as tiff 
from patchify import patchify, unpatchify
from sklearn.model_selection import train_test_split
from patchify import patchify
from sklearn.metrics import jaccard_score,  accuracy_score, f1_score, make_scorer, mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from keras.utils.vis_utils import plot_model
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
import random
#%load_ext tensorboard
from keras import backend as K
np.random.seed=42
from spectral import *
from tensorflow.keras.callbacks import ModelCheckpoint

ModuleNotFoundError: No module named 'keras.utils.vis_utils'

In [31]:
IoUs = []
Fscores = []

In [32]:
HEIGHT = 512
WIDTH = 512
CHANNELS = 10

In [33]:
def jacard_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)
def jacard_coef_loss(y_true, y_pred):
    return -jacard_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function

In [34]:
def average_calc(X_examine, y_examine):
  IoUs = [len(X_examine)]
  total =0
  average =0

  for i in range(0,len(X_examine)):
    test_predict = model.predict(np.expand_dims(X_examine[i],0))
    test_predict = np.asarray(test_predict) 
    IoU = jaccard_score((test_predict>0.5).squeeze(), (y_examine[i]>0).squeeze(), average='micro')
    total =IoU + total
  average = total / len(X_examine)
  
  return average

In [ ]:
# dataset size testing

for z in range(5,87):

    def jacard_coef(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        intersection = K.sum(y_true_f * y_pred_f)
        return (intersection + 1.0) / (K.sum(y_true_f) + K.sum(y_pred_f) - intersection + 1.0)
    def jacard_coef_loss(y_true, y_pred):   
        return -jacard_coef(y_true, y_pred)  # -1 ultiplied as we want to minimize this value as loss function




    print(z)
    X=[]
    y=[]
    Xpath = 0 
    Ximg = 0
    Field = 0
    ypath = 0
    yimg = 0
    Xpatches = 0
    ypatches = 0

    for a in range(1,z):
        Xpath = '/Users/anshugusain/tf-nuclei/content/Stained-40X/Field' + str(a) + '/cube_envi32.hdr'
        Ximg = spectral.open_image(Xpath)


        Field = Ximg.read_bands((38,40,88,93,95,97,100,105,106,110))
        
        ypath = '/Users/anshugusain/tf-nuclei/content/Stained-40X/Field' + str(a) + '/Field' + str(a) + '-mask.tif'
        yimg = tf.keras.preprocessing.image.load_img(ypath, grayscale=True)
            

        Xpatches = patchify(Field,(HEIGHT,WIDTH,CHANNELS),step=HEIGHT)

        yimg = np.asarray(yimg)>0
        ypatches = patchify(yimg, (HEIGHT,WIDTH), step=HEIGHT)

        for m in range(0, Xpatches.shape[0]):
            for l in range(0, Xpatches.shape[1]):
                X.append(Xpatches[m,l,0])
                y.append(ypatches[m,l])
    X = np.array(X)
    y = np.array(y)
    X = (X/X.max())
    y = (y/y.max())
    X_train, X_test, y_train, y_test = train_test_split(X,y ,random_state=42, test_size=0.20, shuffle=True)
    y_train = y_train.astype(X_train.dtype)
    #Building the model

    #Building the model

    inputs = tf.keras.layers.Input((HEIGHT, WIDTH, CHANNELS))
    s = inputs #converts integers from input layers into floating points

    #Contraction Path
                            # feature space  
                            # \/
    c1 = tf.keras.layers.Conv2D(16,(3,3), activation = "relu", kernel_initializer='he_normal', padding = 'same')(s)#kernel initializer is the initial weights used by the neural network (he_normal is a trunkated normal distribution centered on 0)
    c1 = tf.keras.layers.Dropout(.1)(c1)
    c1 = tf.keras.layers.Conv2D(16,(3,3), activation = "relu", kernel_initializer="he_normal", padding = "same")(c1 ) #Padding ensures that the output image has the same dimensions as the input image
    p1 = tf.keras.layers.MaxPooling2D((2,2))(c1) #First maxpooling layer puts the values into a 2x2 matrix, and gets the maximum values

    c2 = tf.keras.layers.Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p1)
    c2 = tf.keras.layers.Dropout(.1)(c2)
    c2 = tf.keras.layers.Conv2D(32,(3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c2)
    p2=tf.keras.layers.MaxPooling2D((2,2))(c2)

    c3= tf.keras.layers.Conv2D(64, (3,3), activation = "relu", kernel_initializer="he_normal", padding="same")(p2)
    c3 = tf.keras.layers.Dropout(.2)(c3)
    c3=tf.keras.layers.Conv2D(64,(3,3), activation = "relu", kernel_initializer="he_normal", padding="same")(c3)
    p3=tf.keras.layers.MaxPooling2D((2,2))(c3)

    c4= tf.keras.layers.Conv2D(128, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(p3)
    c4= tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3,3),activation="relu", kernel_initializer="he_normal", padding="same")(c4)
    p4=tf.keras.layers.MaxPooling2D((2,2))(c4)

    c5=tf.keras.layers.Conv2D(256, (3,3), activation ="relu", kernel_initializer="he_normal", padding="same")(p4)
    c5=tf.keras.layers.Dropout(0.3)(c5)
    c5=tf.keras.layers.Conv2D(256, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c5)


    u9= tf.keras.layers.Conv2DTranspose(128, (2,2), strides=(2,2), padding="same")(c5)
    u9= tf.keras.layers.concatenate([u9,c4])#Concatenates layers u6 and c4 as shown in the Unet architecture
    c9=tf.keras.layers.Conv2D(128, (3,3), activation="relu", kernel_initializer='he_normal', padding="same")(u9)
    c9=tf.keras.layers.Dropout(0.2)(c9)
    c9=tf.keras.layers.Conv2D(128, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    u10= tf.keras.layers.Conv2DTranspose(64, (2,2), strides=(2,2), padding="same")(c9)
    u10= tf.keras.layers.concatenate([u10,c3])#Concatenates layers u6 and c4 as shown in the Unet architecture
    c10=tf.keras.layers.Conv2D(64, (3,3), activation="relu", kernel_initializer='he_normal', padding="same")(u10)
    c10=tf.keras.layers.Dropout(0.2)(c10)
    c10=tf.keras.layers.Conv2D(64, (3,3), activation='relu', kernel_initializer='he_normal', padding='same')(c10)

    u11=tf.keras.layers.Conv2DTranspose(32, (2,2), strides=(2,2), padding='same')(c10)
    u11=tf.keras.layers.concatenate([u11,c2])
    c11=tf.keras.layers.Conv2D(32,(3,3), activation="relu",kernel_initializer="he_normal", padding="same")(u11)
    c11=tf.keras.layers.Dropout(0.2)(c11)
    c11=tf.keras.layers.Conv2D(32, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c11)

    u12=tf.keras.layers.Conv2DTranspose(16, (2,2),strides=(2,2), padding="same")(c11)
    u12=tf.keras.layers.concatenate([u12, c1])
    c12=tf.keras.layers.Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(u12)
    c12=tf.keras.layers.Dropout(0.1)(c12)
    c12=tf.keras.layers.Conv2D(16, (3,3), activation="relu", kernel_initializer="he_normal", padding="same")(c12)


    outputs=tf.keras.layers.Conv2D(1, (1,1), activation="sigmoid")(c12)
    model=tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer=tf.keras.optimizers.legacy.Adam(lr=.001), loss=jacard_coef_loss, metrics=[jacard_coef])

    
    filepath="/Users/anshugusain/tf-nuclei/Models/new_spectral-bestVAL/best_weights.h5"

    checkpoint = ModelCheckpoint(filepath, monitor='val_jacard_coef', verbose=1,save_best_only=True, mode='max')

    callbacks_list = [checkpoint]

    results = model.fit(X_train, y_train, batch_size = 1, epochs=100, shuffle=False, validation_data=(X_test, y_test), callbacks=callbacks_list)
    
    model = tf.keras.models.load_model('/Users/anshugusain/tf-nuclei/Models/new_spectral-bestVAL/best_weights.h5', custom_objects={'jacard_coef_loss': jacard_coef_loss,'jacard_coef':jacard_coef})

    IoUs.append(average_calc(X_test, y_test))
    Fscores.append(average_Fscore(X_test, y_test))

In [44]:
IoUs

[0.7260111557220231,
 0.7291986325338502,
 0.7035214827202259,
 0.7042840692510264,
 0.6639689336469703,
 0.7002385577015999,
 0.6737013300500994,
 0.6607808606542046,
 0.6746150606750018,
 0.702892615503278,
 0.7007684116676335,
 0.6961304163984755,
 0.6607532049251525,
 0.6791955321953307,
 0.6827907817211979,
 0.6906188659499013,
 0.6941138672845243,
 0.6938296828941757,
 0.7054128686407878,
 0.6855396360015489,
 0.7083219793994223,
 0.1385345458984375,
 0.1385345458984375,
 0.12276840209960938,
 0.6851317709358986,
 0.6766309371732583,
 0.12196981855564866,
 0.6743085745493094,
 0.6795130881620262,
 0.12243270874023438,
 0.6976754573170644,
 0.11956890667919579,
 0.6849449400905866]